In [1]:
import pandas as pd
import torchaudio.transforms as T
from pytube import YouTube
import torchaudio
from tqdm.notebook import tqdm
import numpy as np
import glob
from sklearn.model_selection import StratifiedKFold
from pytube.exceptions import VideoPrivate

In [2]:
data = pd.read_csv('wanted.csv')

In [3]:
data

,YTID,start,end,labels
0,-9ek6eO0RtI,260.0,270.0,/m/03wwcy
1,-JKLmqDk9p8,490.0,500.0,"/m/06mb1,/m/07r10fb,/m/0jb2l,/m/0ngt1,/t/dd00038"
2,-gStrDahsjU,20.0,30.0,"/m/03wwcy,/m/04rlf,/m/09x0r,/t/dd00126"
3,-uSlzn9oPQ4,20.0,30.0,"/m/06mb1,/m/0jb2l,/m/0ngt1,/t/dd00038"
4,021B2ozAjNg,110.0,120.0,"/m/01h8n0,/m/09x0r"
...,...,...,...,...
235,yw9MbiKIhdM,120.0,130.0,"/m/01h8n0,/m/09x0r"
236,zQpPfONDPXs,360.0,370.0,"/m/0463cq4,/m/09x0r,/t/dd00002,/t/dd00125"
237,zY1LJu1j4zw,30.0,40.0,/t/dd00002
238,zkNbNA2mLIg,20.0,30.0,/t/dd00002


In [4]:
class_map = {
    '/t/dd00002':'Baby',
    '/m/03wwcy':'Doorbell',
    '/m/01h8n0':'Conversation',
    '/t/dd00038':'Rain'
}

In [13]:
def download_audio(df:pd.DataFrame):
    miss_vids = []
    root_url = "https://www.youtube.com/watch?v="
    n = len(df)
    for i in tqdm(range(n)):
        uid = df.loc[i,'YTID']
        l = df.loc[i,'labels'].split(',')
        if len(l) == 1:
            p = class_map[l[0]]
        else:
            for j in class_map.keys():
                if j in l:
                    p = class_map[j]
        try:
            yt = YouTube(url=root_url+uid)
            title = yt.title
            stream = yt.streams.filter(only_audio=True).all()
            miss_vids.append({'path':f"{p}/{title}",'vides-id':uid,'start_time':df.loc[i,'start'],'end_time':df.loc[i,'end'],'class':p})
            print(yt)
        except (Exception):
            print(f"{uid}")
            pass
        
        else:
            stream[0].download(p)
    return miss_vids        

In [ ]:
missed = download_audio(data)

In [15]:
df = pd.DataFrame(missed)

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
df.to_csv("160.csv",index=False)

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Analysis and Preprocessing

In [18]:
aud_df = pd.read_csv("160.csv")

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
aud_df

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,path,vides-id,start_time,end_time,class
0,Doorbell/Cordless Remote Wireless Door Bell @m...,-9ek6eO0RtI,260.0,270.0,Doorbell
1,Rain/Tibetan Bell & Rain Meditation,-JKLmqDk9p8,490.0,500.0,Rain
2,Rain/poe karen new song 2016,-uSlzn9oPQ4,20.0,30.0,Rain
3,"Conversation/Emma Watson, IGN interview 2001",021B2ozAjNg,110.0,120.0,Conversation
4,Doorbell/バックブザー バックメロディー 後退警報音声,02JEvDAcq7U,10.0,20.0,Doorbell
...,...,...,...,...,...
175,Rain/Rain and thunder sounds,y2A1Pmiu7yw,30.0,40.0,Rain
176,Doorbell/Electric Feel - MGMT .cvrbrthrs.,y9na4SXQfFE,10.0,20.0,Doorbell
177,Doorbell/Double D's Epic Rick Roll Doorbell,yNqiSfX8e3Y,12.0,22.0,Doorbell
178,Baby/Alondra Capitulo 56/4,zQpPfONDPXs,360.0,370.0,Baby


In [20]:
aud_df['offset'] = aud_df['end_time'] - aud_df['start_time']

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
aud_df.to_csv('160.csv',index=False)

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
aud_df['class'].value_counts()

Rain            50
Conversation    44
Baby            44
Doorbell        42
Name: class, dtype: int64

In [28]:
aud_df['folds'] = -1
for i,(train_id,val_id) in enumerate(StratifiedKFold(n_splits=5).split(aud_df.path,aud_df['class'])) :
    #print(i,train_id,val_id)
    aud_df.loc[val_id,'folds'] = i

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
aud_df.folds.value_counts()

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    36
1    36
2    36
3    36
4    36
Name: folds, dtype: int64

In [31]:
aud_df.to_csv('160.csv',index=False)

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
aud_df['duration'] = aud_df['offset']
aud_df.drop(labels=['offset'],axis=1)

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,path,vides-id,start_time,end_time,class,folds,duration
0,Doorbell/Cordless Remote Wireless Door Bell @m...,-9ek6eO0RtI,260.0,270.0,Doorbell,0,10.0
1,Rain/Tibetan Bell & Rain Meditation,-JKLmqDk9p8,490.0,500.0,Rain,0,10.0
2,Rain/poe karen new song 2016,-uSlzn9oPQ4,20.0,30.0,Rain,0,10.0
3,"Conversation/Emma Watson, IGN interview 2001",021B2ozAjNg,110.0,120.0,Conversation,0,10.0
4,Doorbell/バックブザー バックメロディー 後退警報音声,02JEvDAcq7U,10.0,20.0,Doorbell,0,10.0
...,...,...,...,...,...,...,...
175,Rain/Rain and thunder sounds,y2A1Pmiu7yw,30.0,40.0,Rain,4,10.0
176,Doorbell/Electric Feel - MGMT .cvrbrthrs.,y9na4SXQfFE,10.0,20.0,Doorbell,4,10.0
177,Doorbell/Double D's Epic Rick Roll Doorbell,yNqiSfX8e3Y,12.0,22.0,Doorbell,4,10.0
178,Baby/Alondra Capitulo 56/4,zQpPfONDPXs,360.0,370.0,Baby,4,10.0


In [33]:
aud_df.to_csv('160.csv',index=False)

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
aud_df.duration.value_counts()

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


10.0    173
3.0       2
6.0       2
7.0       1
9.0       1
4.0       1
Name: duration, dtype: int64

In [2]:
import librosa

In [64]:
aud_df[aud_df.path == 'Baby/3 Year Old Crying over Justin Beaver']

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,path,vides-id,start_time,end_time,class,offset,folds,duration


In [27]:
y,sr = torchaudio.load("/media/lustbeast/6A38216B38213789/Intern Work/audio tagging/Baby/3 Year Old Crying over Justin Beaver.wav",frame_offset=30*44100,num_frames=10*44100)

In [28]:
sr

44100

In [29]:
y.shape

torch.Size([2, 441000])

In [30]:
y[0]

tensor([-0.0191, -0.0161, -0.0131,  ..., -0.0176, -0.0154, -0.0137])

In [31]:
len(y)

2

In [32]:
mel = T.MelSpectrogram(sample_rate=44100)

/home/lustbeast/.virtualenvs/baseline/lib/python3.8/site-packages/torchaudio/functional/functional.py:432: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [33]:
yy = mel(y)

In [34]:
yy.shape

torch.Size([2, 128, 2206])